# Grasp Bayesian Optimization

The goal is to robustify a grasp that has been shown to the robot using bayesian optimization.

In [1]:
from bayes_opt import BayesianOptimization
from smart_grasping_sandbox.smart_grasper import SmartGrasper
from tf.transformations import quaternion_from_euler
from math import pi
import time
import rospy
from math import sqrt, pow
import random
from sys import argv
from numpy import var, mean


sgs = SmartGrasper()

MIN_LIFT_STEPS = 5
MAX_BALL_DISTANCE = 0.6

REPEAT_GRASP = 10

[INFO] [WallTime: 1504180219.210028] [12867.310000] STARTING CONTROLLERS


## Creating our function to optimize

So we want to optimise the grasp stability function. This function will use the Smart Grasping Sandbox to grasp a ball, shake it and compute the stability of the grasp: the less the distance from the ball to the palm changes, the more stable the grasp.

In [5]:
class GraspQuality(object):
        
    def __init__(self, sgs):
        self.sgs = sgs
        self.last_distance = None
        self.current_grasp = {}

    def check_stable(self, joint_names):
        current_min = 1000
        positions = []
        velocities = []
        efforts = []
        for k in range(30):
            sgs.move_tip(y=0.02)
            ball_distance = self.__compute_euclidean_distance()
            if k > MIN_LIFT_STEPS and ball_distance < current_min:
                current_min = ball_distance
            if ball_distance > MAX_BALL_DISTANCE:
                break
                
            time.sleep(0.01)
        robustness = (1/(current_min - 0.18))**2
        return robustness

    def __compute_euclidean_distance(self):
        ball_pose = self.sgs.get_object_pose()
        hand_pose = self.sgs.get_tip_pose()
        dist = sqrt((hand_pose.position.x - ball_pose.position.x)**2 + \
                     (hand_pose.position.y - ball_pose.position.y)**2 + \
                     (hand_pose.position.z - ball_pose.position.z)**2)
        return dist
    
    def run_experiments(self, grasp_distance,
                        H1_F1J1, H1_F1J2, H1_F1J3,
                        H1_F2J1, H1_F2J2, H1_F2J3,
                        H1_F3J1, H1_F3J2, H1_F3J3):
        robustness = []
        for _ in range(REPEAT_GRASP):
            robustness.append(self.experiment(grasp_distance,
                                              H1_F1J1, H1_F1J2, H1_F1J3,
                                              H1_F2J1, H1_F2J2, H1_F2J3,
                                              H1_F3J1, H1_F3J2, H1_F3J3))
            
        # trying to maximize the robustness average - while minimizing its variance
        utility = mean(robustness) / max(0.001,sqrt(var(robustness))) # don't divide by 0
            
        return utility
            
    def experiment(self, grasp_distance,
                   H1_F1J1, H1_F1J2, H1_F1J3,
                   H1_F2J1, H1_F2J2, H1_F2J3,
                   H1_F3J1, H1_F3J2, H1_F3J3):
        self.sgs.reset_world()
        time.sleep(0.1)
        self.sgs.reset_world()
        time.sleep(0.1)

        self.sgs.open_hand()
        time.sleep(0.1)
        self.sgs.open_hand()
        time.sleep(0.1)

        ball_pose = self.sgs.get_object_pose()
        ball_pose.position.z += 0.5

        #setting an absolute orientation (from the top)
        quaternion = quaternion_from_euler(-pi/2., 0.0, 0.0)
        ball_pose.orientation.x = quaternion[0]
        ball_pose.orientation.y = quaternion[1]
        ball_pose.orientation.z = quaternion[2]
        ball_pose.orientation.w = quaternion[3]

        self.sgs.move_tip_absolute(ball_pose)

        self.sgs.move_tip(y=grasp_distance)

        # close the grasp
        self.sgs.check_fingers_collisions(False)
        
        self.current_grasp["H1_F1J1"] = H1_F1J1
        self.current_grasp["H1_F1J2"] = H1_F1J2
        self.current_grasp["H1_F1J3"] = H1_F1J3
        
        self.current_grasp["H1_F2J1"] = H1_F2J1
        self.current_grasp["H1_F2J2"] = H1_F2J2
        self.current_grasp["H1_F2J3"] = H1_F2J3
        
        self.current_grasp["H1_F3J1"] = H1_F3J1
        self.current_grasp["H1_F3J2"] = H1_F3J2
        self.current_grasp["H1_F3J3"] = H1_F3J3
        
        
        self.sgs.send_command(self.current_grasp, duration=1.0)

        # lift slowly and check the quality
        joint_names = self.current_grasp.keys()

        robustness = self.check_stable(joint_names)

        rospy.loginfo("Grasp quality = " + str(robustness))

        sgs.check_fingers_collisions(True)
        
        return robustness

## Running the Bayesian Optimization

And we can now optimize this black box function.

In [6]:
sgs = SmartGrasper()
grasp_quality = GraspQuality(sgs)

bo = BayesianOptimization(grasp_quality.run_experiments,
                          {"grasp_distance": (-0.165, -0.16),
                           "H1_F1J1": (-0.2, 0.2),
                           "H1_F1J2": (0.0, 0.5),
                           "H1_F1J3": (0.2, 0.6),
                           "H1_F2J1": (-0.2, 0.2),
                           "H1_F2J2": (0.0, 0.5),
                           "H1_F2J3": (0.2, 0.6),
                           "H1_F3J1": (-0.2, 0.2),
                           "H1_F3J2": (0.0, 0.5),
                           "H1_F3J3": (0.2, 0.6)
                          })

[INFO] [WallTime: 1504180384.379762] [13031.070000] STARTING CONTROLLERS


In [7]:
bo.maximize(init_points=15, n_iter=10, kappa=4)



Initialization
------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   H1_F1J1 |   H1_F1J2 |   H1_F1J3 |   H1_F2J1 |   H1_F2J2 |   H1_F2J3 |   H1_F3J1 |   H1_F3J2 |   H1_F3J3 |   grasp_distance | 
[INFO] [WallTime: 1504180386.738352] [13033.318000] STARTING CONTROLLERS
[INFO] [WallTime: 1504180386.947866] [13033.422000] STARTING CONTROLLERS
[INFO] [WallTime: 1504180399.795376] [13046.135000] Grasp quality = 11.360527161
[INFO] [WallTime: 1504180400.803948] [13047.141000] STARTING CONTROLLERS
[INFO] [WallTime: 1504180401.013203] [13047.247000] STARTING CONTROLLERS
[INFO] [WallTime: 1504180413.834020] [13059.926000] Grasp quality = 11.3301113649
    1 | 00m28s |  746.01495 |    0.0223 |    0.0028 |    0.3350 |   -0.0177 |    0.1817 |    0.5527 |    0.1296 |    0.3237 |    0.3247 |          -0.1622 | 
Bayesian Optimization
-----------------------------------

In [8]:
best_grasp = bo.res['max']['max_params']

In [9]:
grasp_quality.experiment(best_grasp["grasp_distance"],
                         best_grasp["H1_F1J1"], best_grasp["H1_F1J2"], best_grasp["H1_F1J3"],
                         best_grasp["H1_F2J1"], best_grasp["H1_F2J2"], best_grasp["H1_F2J3"],
                         best_grasp["H1_F3J1"], best_grasp["H1_F3J2"], best_grasp["H1_F3J3"],
                        )

[INFO] [WallTime: 1504180544.912030] [13189.813000] STARTING CONTROLLERS
[INFO] [WallTime: 1504180545.120841] [13189.919000] STARTING CONTROLLERS
[INFO] [WallTime: 1504180558.061862] [13202.735000] Grasp quality = 11.7715838606


11.771583860585748